In [38]:
import datetime
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

from tqdm.notebook import tqdm
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands


# file_name = './BTCUSDT_1m_2022-06-01-00-00-00_2022-12-31-23_59_00.csv' # 횡보하는 장
file_name = './BTCUSDT_5m_2021-03-03-00-00-00_2022-05-01-23_59_00.csv' # 큰 상승과 큰하락이 존재하는 장
df = pd.read_csv(file_name)
df

,datetime,open,high,low,close,volume
0,2021-03-03 00:00:00,48920.08,49070.00,48913.73,49026.90,742.872
1,2021-03-03 00:05:00,49026.91,49028.13,48932.89,48978.76,573.603
2,2021-03-03 00:10:00,48978.77,48996.71,48714.30,48775.50,1394.559
3,2021-03-03 00:15:00,48775.51,48859.85,48601.31,48647.00,1795.561
4,2021-03-03 00:20:00,48647.00,48728.65,48504.79,48553.33,1647.386
...,...,...,...,...,...,...
122395,2022-05-01 23:35:00,37909.00,37936.50,37883.00,37935.00,686.788
122396,2022-05-01 23:40:00,37934.90,37984.60,37934.90,37964.50,764.031
122397,2022-05-01 23:45:00,37964.40,38012.90,37940.00,38007.50,1232.909
122398,2022-05-01 23:50:00,38007.30,38015.00,37985.00,38004.20,875.940


In [39]:
from ta.momentum import RSIIndicator

df['rsi14'] = RSIIndicator(df['low'], window=14).rsi()
df = df.dropna()
df
# df=df[df['datetime']>='2021-03-10 00:00:00'] # rsi 데이터를 계산할때는 앞의 데이터가 조금더 있어야지 정확하게 계산됨

# df=df.reset_index(drop=True)
# df

,datetime,open,high,low,close,volume,rsi14
13,2021-03-03 01:05:00,48622.34,48670.00,48550.00,48597.22,504.339,37.206053
14,2021-03-03 01:10:00,48597.22,48603.50,48454.89,48506.18,824.760,31.159695
15,2021-03-03 01:15:00,48506.17,48618.22,48430.46,48593.49,794.070,29.819221
16,2021-03-03 01:20:00,48593.49,48695.56,48563.00,48623.51,823.374,43.915807
17,2021-03-03 01:25:00,48623.51,48690.00,48581.20,48681.46,505.048,45.533638
...,...,...,...,...,...,...,...
122395,2022-05-01 23:35:00,37909.00,37936.50,37883.00,37935.00,686.788,42.684617
122396,2022-05-01 23:40:00,37934.90,37984.60,37934.90,37964.50,764.031,54.486885
122397,2022-05-01 23:45:00,37964.40,38012.90,37940.00,38007.50,1232.909,55.457520
122398,2022-05-01 23:50:00,38007.30,38015.00,37985.00,38004.20,875.940,62.963057


In [40]:
df=df[df['datetime']>='2021-03-10 00:00:00'] # rsi 데이터를 계산할때는 앞의 데이터가 조금더 있어야지 정확하게 계산됨

df=df.reset_index(drop=True)
df

,datetime,open,high,low,close,volume,rsi14
0,2021-03-10 00:00:00,54537.54,54556.90,54436.13,54468.88,930.369,54.037227
1,2021-03-10 00:05:00,54469.02,54561.22,54466.00,54480.61,570.266,55.280298
2,2021-03-10 00:10:00,54483.33,54483.34,54249.55,54313.92,1781.970,45.646380
3,2021-03-10 00:15:00,54313.92,54440.00,54285.64,54333.08,845.864,47.295654
4,2021-03-10 00:20:00,54333.08,54384.11,54168.93,54380.20,1340.740,42.775385
...,...,...,...,...,...,...,...
120379,2022-05-01 23:35:00,37909.00,37936.50,37883.00,37935.00,686.788,42.684617
120380,2022-05-01 23:40:00,37934.90,37984.60,37934.90,37964.50,764.031,54.486885
120381,2022-05-01 23:45:00,37964.40,38012.90,37940.00,38007.50,1232.909,55.457520
120382,2022-05-01 23:50:00,38007.30,38015.00,37985.00,38004.20,875.940,62.963057


In [41]:
def checkhl(data_back, data_forward, hl):
    if hl == 'high' or hl == 'High':
        ref = data_back[len(data_back)-1]
        for i in range(len(data_back)-1):
            if ref < data_back[i]:
                return 0
        for i in range(len(data_forward)):
            if ref <= data_forward[i]:
                return 0
        return 1
    if hl == 'low' or hl == 'Low':
        ref = data_back[len(data_back)-1]
        for i in range(len(data_back)-1):
            if ref > data_back[i]:
                return 0
        for i in range(len(data_forward)):
            if ref >= data_forward[i]:
                return 0
        return 1


def pivot(osc, LBL, LBR, highlow):
    left = []
    right = []
    pivots = []
    for i in range(len(osc)):
        pivots.append(0.0)
        if i < LBL + 1:
            left.append(osc[i])
        if i > LBL:
            right.append(osc[i])
        if i > LBL + LBR:
            left.append(right[0])
            left.pop(0)
            right.pop(0)
            if checkhl(left, right, highlow):
                pivots[i - LBR] = osc[i - LBR]
    return pivots

# pivots_low = pivot(data, lbl, lbr, 'low')
# pivots_high = pivot(data, lbl, lbr, 'high')

In [42]:
# rsi 저점 값들을 입력시켜주어야함.
lbR = 5
lbL = 5

pivots_low = pivot(df['rsi14'].tolist(), lbL, lbR, 'low')
print(len(pivots_low))

120384


In [43]:
df['pivots_low'] = pivots_low
df[-200:-150]

,datetime,open,high,low,close,volume,rsi14,pivots_low
120184,2022-05-01 07:20:00,38215.4,38284.6,38135.0,38270.1,1653.277,32.807744,32.807744
120185,2022-05-01 07:25:00,38270.0,38314.4,38238.5,38302.0,1019.630,47.734809,0.000000
120186,2022-05-01 07:30:00,38302.0,38341.7,38285.7,38321.0,634.120,52.876213,0.000000
120187,2022-05-01 07:35:00,38320.9,38331.8,38258.3,38302.2,335.877,49.812814,0.000000
120188,2022-05-01 07:40:00,38302.2,38350.0,38241.5,38350.0,792.427,47.977444,0.000000
120189,2022-05-01 07:45:00,38349.9,38350.0,38300.0,38326.7,322.446,54.292808,0.000000
120190,2022-05-01 07:50:00,38326.7,38333.0,38285.5,38294.6,269.745,52.588705,0.000000
120191,2022-05-01 07:55:00,38294.6,38300.7,38212.5,38234.7,490.836,44.940927,0.000000
120192,2022-05-01 08:00:00,38234.8,38259.7,37850.0,37850.1,7630.808,25.280364,0.000000
120193,2022-05-01 08:05:00,37850.1,37910.2,37560.0,37593.2,9140.243,18.360320,0.000000


In [44]:
rsi_lower = 20

df['pivots_low']=df['pivots_low'].apply(lambda x: x if x<rsi_lower else 0.0)
df[-200:-150]

,datetime,open,high,low,close,volume,rsi14,pivots_low
120184,2022-05-01 07:20:00,38215.4,38284.6,38135.0,38270.1,1653.277,32.807744,0.000000
120185,2022-05-01 07:25:00,38270.0,38314.4,38238.5,38302.0,1019.630,47.734809,0.000000
120186,2022-05-01 07:30:00,38302.0,38341.7,38285.7,38321.0,634.120,52.876213,0.000000
120187,2022-05-01 07:35:00,38320.9,38331.8,38258.3,38302.2,335.877,49.812814,0.000000
120188,2022-05-01 07:40:00,38302.2,38350.0,38241.5,38350.0,792.427,47.977444,0.000000
120189,2022-05-01 07:45:00,38349.9,38350.0,38300.0,38326.7,322.446,54.292808,0.000000
120190,2022-05-01 07:50:00,38326.7,38333.0,38285.5,38294.6,269.745,52.588705,0.000000
120191,2022-05-01 07:55:00,38294.6,38300.7,38212.5,38234.7,490.836,44.940927,0.000000
120192,2022-05-01 08:00:00,38234.8,38259.7,37850.0,37850.1,7630.808,25.280364,0.000000
120193,2022-05-01 08:05:00,37850.1,37910.2,37560.0,37593.2,9140.243,18.360320,0.000000


In [45]:
# 전 pivots_low 와 비교하기 
pivots_low_index_list = list(df[df['pivots_low']!=0.0].index)

df['before_low']=0.0
df['before_pivots_low']=0.0
df['before_index']=-1


for i in range(len(pivots_low_index_list)-1):
    before_index = pivots_low_index_list[i]
    next_index = pivots_low_index_list[i+1]
    df.loc[next_index,'before_low']=df.iloc[before_index]['low']
    df.loc[next_index,'before_pivots_low']=df.iloc[before_index]['pivots_low']
    df.loc[next_index,'before_index']=before_index
df

,datetime,open,high,low,close,volume,rsi14,pivots_low,before_low,before_pivots_low,before_index
0,2021-03-10 00:00:00,54537.54,54556.90,54436.13,54468.88,930.369,54.037227,0.0,0.0,0.0,-1
1,2021-03-10 00:05:00,54469.02,54561.22,54466.00,54480.61,570.266,55.280298,0.0,0.0,0.0,-1
2,2021-03-10 00:10:00,54483.33,54483.34,54249.55,54313.92,1781.970,45.646380,0.0,0.0,0.0,-1
3,2021-03-10 00:15:00,54313.92,54440.00,54285.64,54333.08,845.864,47.295654,0.0,0.0,0.0,-1
4,2021-03-10 00:20:00,54333.08,54384.11,54168.93,54380.20,1340.740,42.775385,0.0,0.0,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...
120379,2022-05-01 23:35:00,37909.00,37936.50,37883.00,37935.00,686.788,42.684617,0.0,0.0,0.0,-1
120380,2022-05-01 23:40:00,37934.90,37984.60,37934.90,37964.50,764.031,54.486885,0.0,0.0,0.0,-1
120381,2022-05-01 23:45:00,37964.40,38012.90,37940.00,38007.50,1232.909,55.457520,0.0,0.0,0.0,-1
120382,2022-05-01 23:50:00,38007.30,38015.00,37985.00,38004.20,875.940,62.963057,0.0,0.0,0.0,-1


## plotly를 통하여 tradingview와 동일하게 적용되는지 테스트해보기

In [46]:
# usdt로 매매할수 있는 btc 수량  (usdt -> btc)
def cal_btc_amount(usdt_balance,cur_price):
    """
    거래시점 기준 usdt 으로 얼마만큼의 btc를 구매할수 있는지 계산
    :params float usdt_balance : 구매하고 싶은 usdt 수량
    :params float cur_price : 현재 시점 BTC 가격
    :return amount ex) 0.012 (단위 BTC)
    :rtype float
    """
    # 소수점 자리에서 내림해야하기 때문에 다음과 같이 구현
    amount = math.floor((usdt_balance*1000)/cur_price)/1000 # 최소 단위가 0.001 BTC이므로
    
    return amount

# btc를 usdt 단위로 변환 (btc -> usdt)
def cal_usdt_amount(btc_amount,cur_price):
    """
    거래시점 기준 usdt 으로 얼마만큼의 btc를 구매할수 있는지 계산
    :params float btc_amount : btc_amount
    :params float cur_price : 현재 시점 BTC 가격
    :return amount ex) 126.156 (단위 usdt)
    :rtype float
    """
    return cur_price*btc_amount

# btc 매매 수량에 따른 수수료 계산 
def cal_trade_fee(trade_type,cur_price,btc_amount):
    """
    거래시점 기준 체결된 BTC 수량의 수수료 구하기
    :params str trade_type : 'market' or 'limit' / 시장가 or 지정가
    :params float cur_price : 현재 시점 BTC(또는 다른 코인) 가격
    :params float btc_amount : btc 보유량
    :return cur_trade_fee ex) BTC/USDT라면 0.00915 (단위 usdt)
    :rtype float
    """
    # 거래수수료 (시장가 : 0.04%, 지정가 : 0.02% , 슬리피지 : 0.01% (여기선 시장가에 슬리피지 고려 0.05% 설정))
    # binance 기준
    
    if trade_type == 'market':
        fee = 0.0004 # 0.04 %
    elif trade_type == 'limit':
        fee = 0.0002 # 0.02 %
    
    usdt = cur_price*btc_amount
    cur_trade_fee = usdt*(fee+0.0001) # 슬리피지 고려
    
    return cur_trade_fee

In [47]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

start = 0 
end = len(df)

df = df[start:end]

fig=make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.3,0.7])

candle = go.Candlestick(name ='ohlc',
                        x=df['datetime'],
                        open=df['open'],
                        high=df['high'],
                        low=df['low'],
                        close=df['close'])

rsi = go.Scatter(name ='rsi',x=df['datetime'], y=df['rsi14'])

fig.add_trace(rsi,row=1,col=1)
fig.add_trace(candle,row=2,col=1)

first_balance = 1000000
balance = first_balance   # 잔고 usdt 기준 1000만원
balance_rate = 1 # 잔고 사용 비율, 2이면 2배 레버리지 사용

balance = balance * balance_rate

revenue_rate = 0.03
loss_rate = 0.03 # loss_rate*100 퍼센트 손실이 발생할때마다 추가매수 진행



revenue_t = 0
open_tot_cnt = 0 # 포지션 오픈 횟수
plus_count = 0 
minus_count = 0

max_of_lookback_range = 300 # 최소 하루안에 다이버전스를 체크해야함 5분봉 12개*24시간 = 288 개 여유롭게 300으로 설정

one_trade_btc = 1 # 한번 오픈할 btc 수량

max_loss_position_open_count = 1 # 추가매수 횟수
open_cnt_limit = 5 # 최대 포지션 오픈 횟수
open_long_cnt = 0  # 롱포지션 오픈 횟수
open_long_list = [] # (btc open 수량 ,open시점 가격,익절가,손절가)

# 현재 타임스텝은 i+5번째를 의미 i+6 open은 i+5의 close를 의미함
# i+5번째 봉이 생성될때까지 원하는 가격에 도달했으면 익절 손절 먼저 진행
for i in tqdm(range(len(df)-6)): # pivots_low를 구하려면 그 기준으로 최소한 5개 오른쪽 봉이 생성이되어야함. 그런다음 봉에서 시가에 포지션 오픈
    open_price = df.iloc[i+6]['open'] # df.iloc[i+5]['close'] 우리는 5번째 봉이 생성되어야지만 5번째 전의 봉이 rsi 저점인지 파악 가능
    
    high_price = df.iloc[i+5]['high']
    low_price = df.iloc[i+5]['low']
    
    # 포지션 클로즈
    if open_long_list: # 현재 오픈중인 포지션이 있을 경우 포지션 클로즈 할수 있는지 체크하기
        
        new_open_long_list=[]
        
        for open_price,revenue_price,loss_price_list in open_long_list: # 평단가, 익절가 ,손절가 리스트
            
            # 현재 포지션의 보유 btc_amount
            open_btc_amount = one_trade_btc*(max_loss_position_open_count+1-len(loss_price_list))
            
            if revenue_price<high_price: # 익절 가격에 도달한 경우 익절
       
                now_revenue = (revenue_price-open_price)* open_btc_amount

                # 익절 가격에 오픈한 btc양에 따른 수수료
                close_trade_fee = cal_trade_fee('market',revenue_price,open_btc_amount)

                # btc를 close함으로써 들어오는 usdt (롱이므로 가격이 상승하면 상승한만큼 더 들어옴)
                # open_price*open_btc_amount + now_revenue
                close_usdt_amount = open_price*open_btc_amount + now_revenue
                balance = balance + close_usdt_amount - close_trade_fee

                revenue_t += now_revenue - close_trade_fee

                open_long_cnt-=1 # 롱포지션 종료
                plus_count+=1
                
                print(f"[{i} || Long Close (+) ] time:{df.iloc[i+5]['datetime']} || close_btc_amount :{open_btc_amount} || close_btc_price : {revenue_price} || close_trade_fee: {close_trade_fee} || balance : {balance}|| now revenue : {now_revenue - close_trade_fee} || total revenue:{revenue_t}")
                print(f"now long open cnt : {open_long_cnt}")
                
                fig.add_annotation(
                    x=df.iloc[i+5]['datetime'],
                    y=revenue_price,
                    text=f'LC_p',
                    row=2,col=1)
  
                continue            
            
            # loss_price_list에 있는 가격들 모두 체크해주어야함.
            while len(loss_price_list)>1: # loss price_list가 한개가 남개 된다면 그것은 이제 손절가만 남은 경우

                loss_price = loss_price_list[0]

                if loss_price>low_price: # 추가 포지션 오픈 진행함

                    # 추가 포지션 오픈후 평단가
                    open_price = ((open_btc_amount*open_price)+(one_trade_btc*loss_price))/(open_btc_amount+one_trade_btc)


                    
                    open_btc_amount = open_btc_amount+one_trade_btc

                    # 익절가격도 그에 따라 변화함 (다시 구한 평단가 대비 수익률)
                    revenue_price = open_price*(1+revenue_rate)

                    # loss_price_list도 업데이트 해주어야함
                    loss_price_list = loss_price_list[1:]
                    
                    # 현재 포지션 보유 btc_amount 업데이트
                    open_btc_amount = one_trade_btc*(max_loss_position_open_count+1-len(loss_price_list))
                    
                    # 수수료 계산
                    open_trade_fee = cal_trade_fee('market',loss_price,one_trade_btc) # 매매한 btc량에 따른 수수료  

                    # 구매한 수량에 따른 usdt
                    open_usdt_amount = cal_usdt_amount(one_trade_btc,loss_price) # 계산된 btc량을 usdt 단위로 변환

                    balance = balance - open_usdt_amount - open_trade_fee

                    revenue_t -= open_trade_fee

                else: # 추가 포지션을 오픈하지 못한 경우 (= 즉, 현재 타임스텝의 고가와 저가에서 포지션 종료를 못함)
                    break

            if len(loss_price_list)>1: # 아직 추가 포지션 오픈 기회가 남은 경우 -> 다음 타임스텝에서 조사해야함
                new_open_long_list.append((open_price,revenue_price,loss_price_list))

            elif len(loss_price_list)==1: # 마지막 가격 까지 도달한경우 -> 손절
                loss_price = loss_price_list[0]

                if loss_price>low_price: #  손절 가격에 도달한 경우 손절

                    now_revenue = (loss_price-open_price)*open_btc_amount # 음수가 나옴

                    # 손절 가격에 오픈한 btc양에 따른 수수료
                    close_trade_fee = cal_trade_fee('market',loss_price,open_btc_amount)

                    # btc를 close함으로써 들어오는 usdt (롱이므로 가격이 하락하면 하락한만큼 덜 들어옴)
                    # open_price*open_btc_amount + now_revenue
                    close_usdt_amount = open_price*open_btc_amount + now_revenue
                    balance = balance + close_usdt_amount - close_trade_fee

                    revenue_t += now_revenue - close_trade_fee # now_revenue가 음수이므로 그대로 더하면됨

                    open_long_cnt-=1 # 롱포지션 종료
                    minus_count +=1
                    
                    print(f"[{i} || Long Close (-) ] time:{df.iloc[i+5]['datetime']} || close_btc_amount :{open_btc_amount} || close_btc_price : {loss_price} || close_trade_fee: {close_trade_fee} || balance : {balance} || now revenue : {now_revenue - close_trade_fee} || total revenue:{revenue_t}")
                    print(f"now long open cnt : {open_long_cnt}")
                    
                    fig.add_annotation(
                        x=df.iloc[i+5]['datetime'],
                        y=loss_price,
                        text=f'LC_m',
                        row=2,col=1)
                    continue

                else: # 마지막 손절 가격만 남았지만 현재타임스텝에서는 아직 손절가까지는 도달하지 않은 경우
                    new_open_long_list.append((open_price,revenue_price,loss_price_list))   

        open_long_list = new_open_long_list
    
    # 포지션 오픈 #
    if open_long_cnt>=open_cnt_limit: # 포지션을 최대 오픈했으면 더이상 포지션 오픈하지 않고 다음 진행
        continue
    
    
    if df.iloc[i]['before_pivots_low']==0.0: # 값이 존재하지 않으면 그대로 패스 (포지션을 더이상 진행안함)
        continue
    
    before_pivots_low = df.iloc[i]['before_pivots_low']
    after_pivots_low = df.iloc[i]['pivots_low']
    before_low = df.iloc[i]['before_low']
    after_low = df.iloc[i]['low']
    before_index = df.iloc[i]['before_index']
    
    if before_pivots_low < after_pivots_low and before_low > after_low and i-before_index<=max_of_lookback_range:
        open_btc_amount = one_trade_btc
        open_usdt_amount = cal_usdt_amount(open_btc_amount,open_price) # 계산된 btc량을 usdt 단위로 변환
        open_trade_fee = cal_trade_fee('market',open_price,open_btc_amount) # 매매한 btc량에 따른 수수료

        balance = balance - open_usdt_amount - open_trade_fee
        open_long_cnt+=1
        
        open_long_list.append((open_price,open_price*(1+revenue_rate),[open_price*(1-(loss_rate*(i+1))) for i in range(max_loss_position_open_count)]))
        
        revenue_t -= open_trade_fee
        open_tot_cnt +=1
                
        print(f"[{i} || Long Open] time:{df.iloc[i+6]['datetime']} || open_btc_amount :{open_btc_amount} || open_btc_price : {open_price} || open_trade_fee: {open_trade_fee} || balance : {balance} || total revenue:{revenue_t}")
        print(f"now long open cnt : {open_long_cnt}")     
        
        fig.add_annotation(
            x=df.iloc[i+6]['datetime'],
            y=open_price,
            text=f'LO',
            row=2,col=1)
        

  0%|          | 0/120378 [00:00<?, ?it/s]

[1548 || Long Open] time:2021-03-15 09:30:00 || open_btc_amount :1 || open_btc_price : 59947.02 || open_trade_fee: 29.973509999999997 || balance : 940023.0064899999 || total revenue:-29.973509999999997
now long open cnt : 1
[1629 || Long Close (-) ] time:2021-03-15 16:10:00 || close_btc_amount :1 || close_btc_price : 58148.609399999994 || close_trade_fee: 29.0743047 || balance : 998142.5415852999 || now revenue : -1827.484904700003 || total revenue:-1857.458414700003
now long open cnt : 0
[1660 || Long Open] time:2021-03-15 18:50:00 || open_btc_amount :1 || open_btc_price : 55768.18 || open_trade_fee: 27.88409 || balance : 942346.4774952999 || total revenue:-1885.342504700003
now long open cnt : 1
[1841 || Long Close (-) ] time:2021-03-16 09:50:00 || close_btc_amount :1 || close_btc_price : 54095.1346 || close_trade_fee: 27.0475673 || balance : 996414.5645279998 || now revenue : -1700.0929673000028 || total revenue:-3585.4354720000056
now long open cnt : 0
[1850 || Long Open] time:2021

In [48]:
print(plus_count,minus_count)


30 36


In [37]:
fig.show()

In [3]:
# usdt로 매매할수 있는 btc 수량  (usdt -> btc)
def cal_btc_amount(usdt_balance,cur_price):
    """
    거래시점 기준 usdt 으로 얼마만큼의 btc를 구매할수 있는지 계산
    :params float usdt_balance : 구매하고 싶은 usdt 수량
    :params float cur_price : 현재 시점 BTC 가격
    :return amount ex) 0.012 (단위 BTC)
    :rtype float
    """
    # 소수점 자리에서 내림해야하기 때문에 다음과 같이 구현
    amount = math.floor((usdt_balance*1000)/cur_price)/1000 # 최소 단위가 0.001 BTC이므로
    
    return amount

# btc를 usdt 단위로 변환 (btc -> usdt)
def cal_usdt_amount(btc_amount,cur_price):
    """
    거래시점 기준 usdt 으로 얼마만큼의 btc를 구매할수 있는지 계산
    :params float btc_amount : btc_amount
    :params float cur_price : 현재 시점 BTC 가격
    :return amount ex) 126.156 (단위 usdt)
    :rtype float
    """
    return cur_price*btc_amount

# btc 매매 수량에 따른 수수료 계산 
def cal_trade_fee(trade_type,cur_price,btc_amount):
    """
    거래시점 기준 체결된 BTC 수량의 수수료 구하기
    :params str trade_type : 'market' or 'limit' / 시장가 or 지정가
    :params float cur_price : 현재 시점 BTC(또는 다른 코인) 가격
    :params float btc_amount : btc 보유량
    :return cur_trade_fee ex) BTC/USDT라면 0.00915 (단위 usdt)
    :rtype float
    """
    # 거래수수료 (시장가 : 0.04%, 지정가 : 0.02% , 슬리피지 : 0.01% (여기선 시장가에 슬리피지 고려 0.05% 설정))
    # binance 기준
    
    if trade_type == 'market':
        fee = 0.0004 # 0.04 %
    elif trade_type == 'limit':
        fee = 0.0002 # 0.02 %
    
    usdt = cur_price*btc_amount
    cur_trade_fee = usdt*(fee+0.0001) # 슬리피지 고려
    
    return cur_trade_fee

In [ ]:
def run_test(config):
    first_balance = 10000
    balance = first_balance   # 잔고 usdt 기준 1000만원
    balance_rate = 1 # 잔고 사용 비율, 2이면 2배 레버리지 사용
    
    balance = balance * balance_rate
    
    for i in tqdm(range(len(df))):